In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from scipy.stats import chi2_contingency, beta
from IPython.display import Image

In [2]:
df = pd.read_csv(r"./ab_data.csv")
df.head(10)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


In [3]:
start_time = dt.datetime.strptime(df['timestamp'].min(), '%Y-%m-%d %H:%M:%S.%f')
end_time = dt.datetime.strptime(df['timestamp'].max(), '%Y-%m-%d %H:%M:%S.%f')
duration = (end_time - start_time).days

In [4]:
print('number of unique user experiment: {}'.format(df['user_id'].nunique()))
print('data collected for:{} days'.format(duration))
print('Landing page to compare: {}'.format(df['landing_page'].unique().tolist()))
print('Number of user in control and treatment:{}'.format(df['group'].value_counts()))
print('Percentage of control group:{} %'.format(round(df[df["group"]=="control"].shape[0]/df.shape[0]*100)))

number of unique user experiment: 290584
data collected for:21 days
Landing page to compare: ['old_page', 'new_page']
Number of user in control and treatment:group
treatment    147276
control      147202
Name: count, dtype: int64
Percentage of control group:50 %


In [5]:
sample = df[df['user_id'].isin([746755,722274])]
sample

,user_id,timestamp,group,landing_page,converted
29073,746755,2017-01-11 01:28:57.083669,control,new_page,1
105487,722274,2017-01-19 01:46:53.093257,control,old_page,0
262554,722274,2017-01-09 21:21:23.638444,control,new_page,0
286566,746755,2017-01-05 03:40:08.457451,control,old_page,0


some users have seen both type of the landing page that's why we need to do some cleaning.
there are 2 options for this:
1. only take the first exposure and delete the second
2. delete the cases altogether

In [6]:
#let's go with the first route, let's get the timestamp from the first exposure

first_exposure = sample.groupby('user_id')['timestamp'].min().to_frame().reset_index()
sample = sample.merge(first_exposure, on =['user_id','timestamp'])
sample

,user_id,timestamp,group,landing_page,converted
0,722274,2017-01-09 21:21:23.638444,control,new_page,0
1,746755,2017-01-05 03:40:08.457451,control,old_page,0


In [7]:
counter = df['user_id'].value_counts()
(counter > 1).value_counts()

count
False    286690
True       3894
Name: count, dtype: int64

there are 3,894 users who are both having old landing page and new landing page. Given that it's only amount to 1.34% of total user, it should be okay to remove these users

In [8]:
#remove user with multiple bucket
valid_user = pd.DataFrame(counter[counter == 1].index, columns =['user_id'])
df = df.merge(valid_user, on=['user_id'])

In [9]:
#add week column to see the data as you would during experiment

df['week'] = df['timestamp'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').isocalendar()[1])
df.head(10)

,user_id,timestamp,group,landing_page,converted,week
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,3
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,2
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,2
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,3
5,936923,2017-01-10 15:20:49.083499,control,old_page,0,2
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1,3
7,719014,2017-01-17 01:48:29.539573,control,old_page,0,3
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1,2


In [10]:
df['week'].value_counts().sort_index()

week
1    83745
2    91380
3    91056
4    20509
Name: count, dtype: int64

### Experiment: Frequentist Approach

In [11]:
#Get stats 
num_weeks = 4
exp_df = df[df['week'] <= num_weeks]
control = exp_df[exp_df['group']=='control']
treatment = exp_df[exp_df['group']=='treatment']

control_conversion_perc = round(control['converted'].sum()/control['converted'].count()*100, 2)
treatment_conversion_perc = round(treatment['converted'].sum()/treatment['converted'].count()*100, 2)
uplift = round(treatment_conversion_perc - control_conversion_perc, 2)

print("Treatment conversion rate: {} %".format(treatment_conversion_perc))
print("Control conversion rate: {} %".format(control_conversion_perc))
print("Uplift: {} % ".format(uplift))


Treatment conversion rate: 11.87 %
Control conversion rate: 12.02 %
Uplift: -0.15 % 


# Chi_Squared Test

H0: Control & Treatment are independent
H1: Control & Treatment are not independent

In [13]:
#Create contingecy table for Chi squared test

control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted
contingency_table = np.array([[control_converted, control_non_converted], 
                              [treatment_converted, treatment_non_converted]])

contingency_table


array([[ 17220, 126073],
       [ 17025, 126372]])

In [14]:
chi, p_value, _, _ = chi2_contingency(contingency_table, correction = False)

In [15]:
chi, p_value

(1.426794609399621, 0.23228827305833816)

Since the p_value > 0.05, we cannot reject null hypothesis. Hence, we cannot conclude if there exists a relationship between the control and treatment groups.

In [21]:
formatted_string = f"{round(p_value * 100, 2)}% probability that a more extreme chi square than {round(chi, 3)} would have occurred by chance due to random error."
print(formatted_string)

23.23% probability that a more extreme chi square than 1.427 would have occurred by chance due to random error.


But this is tough to interpret. We would to say something about the actual maginitude of lift. Something like this:

In [24]:
print(f"(We CANNOT say this) We are {round(p_value * 100, 2)}% confident that our lift = {uplift}%")

(We CANNOT say this) We are 23.23% confident that our lift = -0.15%


Bayesian Approach
We want to input the prior distribution and have the experiment update the parameters to create posterier distributions. Since these prior & posterior distributions will be used to sample Conversion Rate, we model them after beta distribtion.

Let's create the prior beta distribtion from the first weeks of conversion data


In [25]:
prior = df[(df['week'] == 1) & (df['group']=='control')]

In [27]:
prior_means = []
for i in range(10000):
    prior_means.append(prior.sample(1000)['converted'].mean())

In [28]:
prior_means[:10]

[0.128, 0.118, 0.123, 0.118, 0.115, 0.122, 0.117, 0.131, 0.135, 0.118]

In [29]:
# Model Beta Distribtion from sample means
prior_alpha, prior_beta, _, _ = beta.fit(prior_means, floc=0, fscale=1)

In [30]:
# Get Stats
NUM_WEEKS = 4 # Vary number to get experiment data at weekly points in time
experiment_data = df[(df['week'] > 1) & (df['week'] <= NUM_WEEKS)]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round((treatment_conversion_perc - control_conversion_perc) / control_conversion_perc , 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.909%
Control Conversion Rate: 12.058%
Lift = -0.012%


In [31]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted

# Update Prior parameters with experiment conversion rates
posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# Sample from Posteriors
control_samples = posterior_control.rvs(1000)
treatment_samples = posterior_treatment.rvs(1000)
probability = np.mean(treatment_samples > control_samples)
print(f"Probability that treatment > control: {probability * 100}%")

Probability that treatment > control: 13.4%


In [33]:
(control_mu), (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()
print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}") 
print(f"Treatment Posterior: Mean: {treatment_mu}, Variance: {treatment_var}") 

Control Posterior: Mean: 0.12056242662969856, Variance: 1.0333507387103082e-06
Treatment Posterior: Mean: 0.1190938488762046, Variance: 1.0242548787439197e-06


We can even make statements like the following which are actionable:

In [34]:
lift_percentage = (treatment_samples - control_samples) / control_samples
print(f"Probability that we are seeing a 2% lift: {np.mean((100 * lift_percentage) > 2) * 100}%")

Probability that we are seeing a 2% lift: 0.3%


Advantages of Bayesian over Frequentist:

1. Results are more interpretable than the ones we got from the frequentist approach
2. 
We can interpret results at any point during the experiment. Don't need to wait for an arbitrary "statsig"

source: https://www.youtube.com/watch?v=OVgi6ftJiyQ&t=1127s